# YAPAY SİNİR AĞLARI DERSİ VİZE UYGULAMA SINAVI PROJESİ

## KONU: YSA ile Kalp Yetmezliği Tahmini

### ADI SOYADI: Enes Yüksel

### NUMARA: 194410044

### BÖLÜM: Bilgisayar Mühendisliği

### SINIF: 4. Sınıf

### TARİH: 27.12.2022

Kardiyovasküler hastalıklar (KVH'ler), her yıl tahmini 17,9 milyon can alarak dünya çapında 1 numaralı ölüm nedenidir ve bu, dünya çapındaki tüm ölümlerin %31'inden sorumludur.
Kalp yetmezliği, CVD'lerin neden olduğu yaygın bir olaydır ve bu veri seti, kalp yetmezliğine bağlı ölümleri tahmin etmek için kullanılabilecek 12 özellik içerir.

12 Özellik: age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,
serum_creatinine,serum_sodium,sex,smoking,time

Çoğu kardiyovasküler hastalık, toplum çapında stratejiler kullanılarak tütün kullanımı, sağlıksız beslenme ve obezite, fiziksel hareketsizlik ve alkolün zararlı kullanımı gibi davranışsal risk faktörleri ele alınarak önlenebilir.

Kardiyovasküler hastalığı olan veya yüksek kardiyovasküler risk altında olan kişiler (hipertansiyon, diyabet, hiperlipidemi veya önceden belirlenmiş hastalık gibi bir veya daha fazla risk faktörünün varlığından dolayı) erken tespit ve yönetime ihtiyaç duyar ve bu durumda bir makine öğrenimi modeli çok yardımcı olabilir.

Kaggle üzerinden, "heart_failure_clinical_records_dataset" isimli Yapay Sinir Ağları ile uyumlu olan veri seti, csv formatıyla indirilmiştir. Bu veri setinin seçilmesinin sebebi, kalp yetmezliği konusunda tamamen nitelikli sayısal veriler içeriyor olmasıdır.

Adım 1: İleriki işlemlerde kullanılacak olan Numpy, Pandas, TensorFlow kütüphaneleri tanımlanır

In [323]:
import numpy as np
import pandas as pd
import tensorflow as tf

Adım 2: "heart_failure_clinical_records_dataset" isimli veri seti okutulur

In [324]:
data = pd.read_csv("heart_failure_clinical_records_dataset.csv")

In [325]:
data.head(10)

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1
5,90.0,1,47,0,40,1,204000.00,2.1,132,1,1,8,1
6,75.0,1,246,0,15,0,127000.00,1.2,137,1,0,10,1
7,60.0,1,315,1,60,0,454000.00,1.1,131,1,1,10,1
8,65.0,0,157,0,65,0,263358.03,1.5,138,0,0,10,1
9,80.0,1,123,0,35,1,388000.00,9.4,133,1,1,10,1


Adım 3: Tüm bağımsız değişkenlerimizi içeren Özellikler Matrisi oluşturulur.

(Burada ".iloc" ile sayısal değer belirterek işlem yapabiliriz, buna alternatif olarak ".drop" ile belirlenen string ile de özellikler matrisini oluşturabiliriz)

In [326]:
#X = data.iloc[:,0:-1].values
X = data.drop(["DEATH_EVENT"], axis=1)

Adım 4: Bağımlı değişken değerlerimizi içerecek bir bağımlı değişken vektörü oluştururuz

(Yukarıdaki ile aynı şekilde ister ".iloc" ile ister ".drop" ile bağımlı değişken vektörü oluşturabiliriz)

In [327]:
#y = data.iloc[:,-1].values
y = data["DEATH_EVENT"]

Adım 5: Veri setimizi, eğitim aşamasında verinin yüzde 80'i oranda olacak ve verinin yüzde 20'si test aşamasında olacak şekilde eğitim ve test veri setlerine ayırırız


"random_state", datayı her seferinde farklı yerlerinden böler. Belirlenen random_state değeri, her seferinde o değere göre böler

In [328]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=100)

In [329]:
#from sklearn.model_selection import train_test_split
#X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=0)

Adım 6: Aşırı yüksek değerli değişkenlerin, aşırı düşük değerli değişkenlere hakim olma ve düşük değerli değişkenlerin model tarafından ihmal edilme olasılığı için tüm değişkenlerin aynı ölçeğe dönüştürülmesi gerekir. Dağılımdan bağımsız olarak herhangi bir veri kümesi için kullanılabilen evrensel bir teknik olan Standardizasyon kullanılır

In [330]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Adım 7: "Sequential" Keras sınıfı kullanılarak YSA başlatılır

In [331]:
ann = tf.keras.models.Sequential()

Adım 8: En uygun değer olan 6'yı alan "units" ve hangi aktivasyon fonksiyonunun kullanılacağını belirten "activation" girişleri tanımlanır

In [332]:
ann.add(tf.keras.layers.Dense(units=6,activation="relu"))

Adım 9: İki gizli katman oluşturulacağından aynı adım tekrarlanır

In [333]:
ann.add(tf.keras.layers.Dense(units=6,activation="relu"))

Adım 10: Devamındaki işlemlerde Dense sınıfı kullanılarak, çıktı olarak("DEATH_EVENT") sadece iki sınıfa (1 ve 0) sahip olacağımız bir ikili sınıflandırma yapacak bir nöron tahsis edilir ve ikili sınıflandırma için sigmoid aktivasyon fonksiyonu kullanılır

In [334]:
ann.add(tf.keras.layers.Dense(units=1,activation="sigmoid"))

Adım 11: Deney sonuçları için kritik öneme sahip "optimizer", "loss", "metrics" girdileri ayarlanır ve YSA derlenir

In [335]:
ann.compile(optimizer="adam",loss="binary_crossentropy",metrics=['accuracy'])

#Adım 12: Alternatif "rmsprop" optimizer değeri ayarlanır ve YSA tekrar derlenir

In [336]:
#ann.compile(optimizer="rmsprop",loss="binary_crossentropy",metrics=['accuracy'])

Adım 13: YSA oluşturma sürecindeki son adım olan fitting yöntemi tanımlanır. "X_train", "Y_train", "batch_size" ve "epochs" girdileri ayarlanır

In [337]:
ann.fit(X_train,y_train,batch_size=32,epochs=1000)

Epoch 1/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.7509 - accuracy: 0.5356
Epoch 2/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.7351 - accuracy: 0.5649
Epoch 3/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.7219 - accuracy: 0.5774
Epoch 4/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.7106 - accuracy: 0.5900
Epoch 5/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.6993 - accuracy: 0.5983
Epoch 6/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.6891 - accuracy: 0.6234
Epoch 7/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.6791 - accuracy: 0.6234
Epoch 8/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.6696 - accuracy: 0.6318
Epoch 9/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.6609 - accuracy: 0.6402
Epoch 10/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.6521 - accuracy: 0.6569
Epoch 11/1000
8/8 [

8/8 [==============================] - 0s 1ms/step - loss: 0.3891 - accuracy: 0.8494
Epoch 84/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.3860 - accuracy: 0.8452
Epoch 85/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.3831 - accuracy: 0.8452
Epoch 86/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.3803 - accuracy: 0.8410
Epoch 87/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.3770 - accuracy: 0.8452
Epoch 88/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.3740 - accuracy: 0.8452
Epoch 89/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.3717 - accuracy: 0.8452
Epoch 90/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.3686 - accuracy: 0.8452
Epoch 91/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.3658 - accuracy: 0.8452
Epoch 92/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.3632 - accuracy: 0.8410
Epoch 93/1000
8/8 [=====

8/8 [==============================] - 0s 1ms/step - loss: 0.2798 - accuracy: 0.8703
Epoch 166/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.2792 - accuracy: 0.8787
Epoch 167/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.2784 - accuracy: 0.8745
Epoch 168/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.2778 - accuracy: 0.8787
Epoch 169/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.2770 - accuracy: 0.8787
Epoch 170/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.2759 - accuracy: 0.8828
Epoch 171/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.2753 - accuracy: 0.8828
Epoch 172/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.2744 - accuracy: 0.8828
Epoch 173/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.2735 - accuracy: 0.8745
Epoch 174/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.2726 - accuracy: 0.8745
Epoch 175/1000


8/8 [==============================] - 0s 1000us/step - loss: 0.2299 - accuracy: 0.8954
Epoch 248/1000
8/8 [==============================] - 0s 1000us/step - loss: 0.2295 - accuracy: 0.8996
Epoch 249/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.2291 - accuracy: 0.8954
Epoch 250/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.2285 - accuracy: 0.8996
Epoch 251/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.2281 - accuracy: 0.8996
Epoch 252/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.2278 - accuracy: 0.8996
Epoch 253/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.2272 - accuracy: 0.8996
Epoch 254/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.2267 - accuracy: 0.8996
Epoch 255/1000
8/8 [==============================] - 0s 1000us/step - loss: 0.2263 - accuracy: 0.9038
Epoch 256/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.2256 - accuracy: 0.9038
Epoch 

Epoch 329/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.1922 - accuracy: 0.9163
Epoch 330/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.1914 - accuracy: 0.9163
Epoch 331/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.1906 - accuracy: 0.9163
Epoch 332/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.1906 - accuracy: 0.9163
Epoch 333/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.1900 - accuracy: 0.9163
Epoch 334/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.1896 - accuracy: 0.9163
Epoch 335/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.1894 - accuracy: 0.9163
Epoch 336/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.1889 - accuracy: 0.9163
Epoch 337/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.1888 - accuracy: 0.9163
Epoch 338/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.1883 - accuracy: 0.9163


...

8/8 [==============================] - 0s 1ms/step - loss: 0.1145 - accuracy: 0.9540
Epoch 656/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.1145 - accuracy: 0.9540
Epoch 657/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.1143 - accuracy: 0.9540
Epoch 658/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.1138 - accuracy: 0.9540
Epoch 659/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.1137 - accuracy: 0.9540
Epoch 660/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.1135 - accuracy: 0.9540
Epoch 661/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.1133 - accuracy: 0.9540
Epoch 662/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.1132 - accuracy: 0.9540
Epoch 663/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.1128 - accuracy: 0.9540
Epoch 664/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.1129 - accuracy: 0.9540
Epoch 665/1000


8/8 [==============================] - 0s 1ms/step - loss: 0.1014 - accuracy: 0.9665
Epoch 738/1000
8/8 [==============================] - 0s 1000us/step - loss: 0.1012 - accuracy: 0.9623
Epoch 739/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.1009 - accuracy: 0.9665
Epoch 740/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.1008 - accuracy: 0.9623
Epoch 741/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.1007 - accuracy: 0.9623
Epoch 742/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.1006 - accuracy: 0.9623
Epoch 743/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.1006 - accuracy: 0.9623
Epoch 744/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.1003 - accuracy: 0.9665
Epoch 745/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.1004 - accuracy: 0.9623
Epoch 746/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.1000 - accuracy: 0.9665
Epoch 747/10

8/8 [==============================] - 0s 1ms/step - loss: 0.0891 - accuracy: 0.9707
Epoch 820/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.0887 - accuracy: 0.9707
Epoch 821/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.0887 - accuracy: 0.9707
Epoch 822/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.0887 - accuracy: 0.9707
Epoch 823/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.0884 - accuracy: 0.9707
Epoch 824/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.0883 - accuracy: 0.9707
Epoch 825/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.0886 - accuracy: 0.9707
Epoch 826/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.0883 - accuracy: 0.9707
Epoch 827/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.0880 - accuracy: 0.9707
Epoch 828/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.0877 - accuracy: 0.9707
Epoch 829/1000


8/8 [==============================] - 0s 1ms/step - loss: 0.0787 - accuracy: 0.9707
Epoch 902/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.0785 - accuracy: 0.9707
Epoch 903/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.0786 - accuracy: 0.9707
Epoch 904/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.0784 - accuracy: 0.9707
Epoch 905/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.0784 - accuracy: 0.9707
Epoch 906/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.0780 - accuracy: 0.9707
Epoch 907/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.0779 - accuracy: 0.9707
Epoch 908/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.0777 - accuracy: 0.9707
Epoch 909/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.0777 - accuracy: 0.9707
Epoch 910/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.0774 - accuracy: 0.9707
Epoch 911/1000


8/8 [==============================] - 0s 1ms/step - loss: 0.0689 - accuracy: 0.9707
Epoch 984/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.0688 - accuracy: 0.9707
Epoch 985/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.0687 - accuracy: 0.9707
Epoch 986/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.0688 - accuracy: 0.9707
Epoch 987/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.0684 - accuracy: 0.9707
Epoch 988/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.0683 - accuracy: 0.9707
Epoch 989/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.0681 - accuracy: 0.9665
Epoch 990/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.0682 - accuracy: 0.9665
Epoch 991/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.0681 - accuracy: 0.9665
Epoch 992/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.0680 - accuracy: 0.9665
Epoch 993/1000


In [338]:
#ann.fit(X_train,y_train,batch_size=128,epochs=100)

In [339]:
#ann.fit(X_train,y_train,batch_size=64,epochs=100)

In [340]:
#ann.fit(X_train,y_train,batch_size=32,epochs=100)

In [341]:
#ann.fit(X_train,y_train,batch_size=32,epochs=500)

In [342]:
#ann.fit(X_train,y_train,batch_size=32,epochs=1000)

Adım 14: Sinir ağı, bağımsız değişkenlerin değerlerine dayanarak müşterinin çıkıp çıkmayacağını tek noktalı olarak tahmin etmeye çalışır

In [343]:
print(ann.predict(sc.transform([[60,1,607,0,40,0,216000,0.6,138,1,1,54]])) > 0.5)

1/1 [==============================] - 0s 42ms/step
[[False]]


### Yapılan deneylerden gelen sonuç değerleri ve yorumlar, proje klasör dizininde yer alan "DeneyGözlemRaporu_194410044.pdf" dosyasında yer almaktadır. Teşekkür ederim.